From the [PVSC44 TL sensitivity](PVSC44%20TL%20sensitivity.ipynb) we concluded that:
    
* Overal MACC data is higher than corresponding static or optimized $T_L$ which leads to low dyanamic predictions.
* For at least 3 SURFRAD stations: bon, psu and sxf high $T_L$ in summer caused a seasonal bias in GHI predictions.
* For the high elevation stations: tbl and dra, that also have very high DNI and low DHI, optimization of $T_L$ led
  to unphysical values, indicating that another factor may, such as sensor error, may be affecting predictions more than
  atmospheric conditions.